<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inversion-as-Quadratic-Programming-Problem" data-toc-modified-id="Inversion-as-Quadratic-Programming-Problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inversion as Quadratic Programming Problem</a></span><ul class="toc-item"><li><span><a href="#Formulation-of-the-Problem" data-toc-modified-id="Formulation-of-the-Problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Formulation of the Problem</a></span></li><li><span><a href="#FEM-Forward-Model" data-toc-modified-id="FEM-Forward-Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>FEM Forward Model</a></span><ul class="toc-item"><li><span><a href="#FEM-domain" data-toc-modified-id="FEM-domain-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>FEM domain</a></span></li><li><span><a href="#Definition-of-indicator-functions" data-toc-modified-id="Definition-of-indicator-functions-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Definition of indicator functions</a></span></li><li><span><a href="#Create-a-density-distribution" data-toc-modified-id="Create-a-density-distribution-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Create a density distribution</a></span></li><li><span><a href="#Compute-the-gravity-field" data-toc-modified-id="Compute-the-gravity-field-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Compute the gravity field</a></span></li><li><span><a href="#Grab-the-data-along-a-line" data-toc-modified-id="Grab-the-data-along-a-line-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Grab the data along a line</a></span></li><li><span><a href="#Forward-model-in-a-compact-form" data-toc-modified-id="Forward-model-in-a-compact-form-1.2.6"><span class="toc-item-num">1.2.6&nbsp;&nbsp;</span>Forward model in a compact form</a></span></li></ul></li><li><span><a href="#Critical-Point-Condition" data-toc-modified-id="Critical-Point-Condition-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Critical Point Condition</a></span><ul class="toc-item"><li><span><a href="#Jacobean-for-the-gravity-problem" data-toc-modified-id="Jacobean-for-the-gravity-problem-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Jacobean for the gravity problem</a></span></li><li><span><a href="#The-linear-model-case" data-toc-modified-id="The-linear-model-case-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>The linear model case</a></span></li></ul></li><li><span><a href="#Solution" data-toc-modified-id="Solution-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Solution</a></span><ul class="toc-item"><li><span><a href="#First-Attempt" data-toc-modified-id="First-Attempt-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>First Attempt</a></span></li><li><span><a href="#SVD" data-toc-modified-id="SVD-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>SVD</a></span></li><li><span><a href="#Some-Analysis" data-toc-modified-id="Some-Analysis-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Some Analysis</a></span></li></ul></li><li><span><a href="#Solution-Strategies" data-toc-modified-id="Solution-Strategies-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Solution Strategies</a></span><ul class="toc-item"><li><span><a href="#Mode-dropping" data-toc-modified-id="Mode-dropping-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Mode dropping</a></span></li><li><span><a href="#Filtering" data-toc-modified-id="Filtering-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Filtering</a></span></li></ul></li><li><span><a href="#Regularization" data-toc-modified-id="Regularization-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Regularization</a></span></li><li><span><a href="#L-curves" data-toc-modified-id="L-curves-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>L-curves</a></span></li></ul></li></ul></div>

by Lutz Gross, The University of Queensland, Australia
<a href="mailto:l.gross@uq.edu.au">l.gross@uq.edu.au</a>
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

We will use this at one point in time 

In [1]:
#%matplotlib notebook # commmend for interactive plots
import matplotlib.pyplot as plt
import numpy as np

mGal=100000
G=6.67e-11  # m^3/kg/sec^2 gravity constant

# this makes the random number generation reproducable: 
np.random.seed(seed=9)

# Inversion as Quadratic Programming Problem
## Formulation of the Problem

**Problem:** How to fit geophysical properties to known observation?

For instance: finding the density anomaly creating a given vertical gravity profile along a transect. 

In mathematical terms this task is state as an optimization problem: 

- given observations $d_i^{obs}$ for observation points $i$ with $i=0\ldots,N_d-1$.
- given a forward model $F_i(\mathbf{m})$ providing a prediction $d_i$ for the observations given a realization $\mathbf{m}$ of the unknown geophysical properties.

**Task:** Find $\mathbf{m}^*$ that minimizes the difference between $d_i = F_i(\mathbf{m})$ and $d_i^{obs}$.

Obviously this needs to be formalized more.


The most common form of measuring misfit is 
a quadratic form $\Phi_d$ in the form 
\begin{equation}\label{eq:misfit}
\Phi_d(\mathbf{m}) = \frac{1}{2} \sum_{i=0}^{N_d-1} w^{obs}_i \cdot |F_i(\mathbf{m})-d_i^{obs}|^2
\end{equation}
where $w^{obs}_k$ are positive weighting factors allowing for adjustments of 
contributions of measurement to the misfit.

When measurement errors $\sigma_i$ are available one can set 
\begin{equation}\label{eq:misfit2}
\Phi_d(\mathbf{m}) = \frac{1}{2} \sum_{i=0}^{N_d-1} \frac{1}{\sigma_i^2} \cdot |F_i(\mathbf{m})-d_i^{obs}|^2
\end{equation}
So we choose in \eqref{eq:misfit}:
\begin{equation}\label{eq:misfit3}
 w^{obs}_i  = \frac{1}{N_d} \frac{1}{\sigma_i^2}
\end{equation}
For this setting
- $\Phi_d(\mathbf{m})<1$ means that in average prediction $d_k$ are a better match of the data $d_k^{obs}$ than the average error This case is referred to as over fitting. 
- $\Phi_d(\mathbf{m})>1$ indicates that data prediction are worse than the error and more work should be done to improve the misfit. This case is referred to as under fitting.
    
    


The misfit is sometimes written in matrix form:
\begin{equation}\label{eq:misfit4}
\Phi_d(\mathbf{m}) = \frac{1}{2} (\mathbf{d}-\mathbf{d}^{obs})^T \mathbf{W}_d (\mathbf{d}-\mathbf{d}^{obs})
\end{equation}
where the prediction and observation vectors are given as
\begin{align}\label{eq:misfit4.1}
\mathbf{d} = & [d_0, \ldots, d_{N_d-1}]^T   \\
\mathbf{d}^{obs} = &  [d^{ops}_0, \ldots, d^{ops}_{N_d-1}]^T
\end{align}



The data weighting matrix $\mathbf{W}_d$ is an $N_d \times N_d$, diagonal matrix it is given as 
\begin{equation}\label{eq:misfit4.2}
\mathbf{W}_d =
\begin{bmatrix}
w^{obs}_0  & 0        &            & 0\\
    0      &  w^{obs}_1&           & 0\\
    \vdots       &        & \ddots &    \vdots\\
        0         &       &       & w^{obs}_{N_d-1}
\end{bmatrix}
\end{equation}


*Notes*
- Misfit $\Phi_d$ \eqref{eq:misfit4} in case of complex valued data (eg. for MT) transposed $T$ needs to 
be replaced by the Hermitian transpose $H$. 
- $\mathbf{W}_d$ may have a more general form but is required to be 
    - symmetric: $\mathbf{W}_d =  \mathbf{W}_d^T$ and 
    - positive definite: $\mathbf{d}^T \mathbf{W}_d \mathbf{d} > 0$ for any non-zero vector $\mathbf{d}$ These properties guaranty that the misfit has non-negative real values. 

Inversion problem is given as a minimization problem, namely to find the vector $\mathbf{m}^{*}$ of $N_p$ components as solution of  
\begin{equation}\label{eq:minimization}
\underset{\mathbf{m}}{\mbox{minimize}} \; \Phi_d(\mathbf{m})
\end{equation}

For a two unknwons $\mathbf{m}=(m_0, m_1)$ the misfit function could like like this:

<img src="./Data/Image_minima.png" alt="MT domain" width="500"> 
 from https://medium.com/analytics-vidhya/journey-of-gradient-descent-from-local-to-global-c851eba3d367

In order to solve this problem we first need to have a mean to evaluate the forward problem(s) $d_i = F_i(\mathbf{m})$ which will do using the finite element method.

## FEM Forward Model

We are looking at the gravity anomaly case:
    
Lets assume the shape of the density anomaly is known but its density $m_0=\rho_0$ is unknown.     

To describe the geometry we use a so called in indicator function $\chi$ which is takes the value 
one for locations with in the anomaly and zero outside, see Figure~\ref{fig:indicator_function}. 
Formally this is expressed as 
\begin{equation}\label{eq:indicator}
\chi(\mathbf{x}) = \left\{ 
\begin{array}{cl}
1 & \mathbf{x} \mbox{ locate inside the anomaly } \\
0 & \mbox{ otherwise }
\end{array}
\right.
\end{equation}
<img src="./Data/IMAGE_Indicator_function.png" alt="MT domain" width="300"> 
With this setting a density distribution $\rho$ is space is defined as 
\begin{equation}\label{eq:prop0}
\rho = m_0 \cdot \chi
\end{equation}
which then defines the right hand side of the PDE we need to solve. 
The property vector $\mathbf{m}=[m_0]$ has one entry only that defines the density in the anomaly.


This concept can be extended by using a set of $N_p$ anomalies each with constant density $m_k$. Then we 
set
\begin{equation}\label{eq:prop1}
\rho = \sum_{k=0}^{N_p-1} m_k \chi_{k}
\end{equation}
Typically one would choose the anomaly non over-lapping but this is not required.
When the functions $\chi_{k}$ are in fact indicator functions then this approach for $\rho$
also referred to as piecewise constant.

A general anomaly can be approximated using patches in the form of square or rectangular patches in an $4 \times 3$ array which is aligned with the FEM grid. Here we use eight patches  (with six observation points $N_d=6$) 
over a $29 \times 18$ computational grid for solving the forward PDE. 
<img src="./Data/IMAGE_fitting_grid.png" alt="MT domain" width="400"> 
Notice that grid lines are aligned with patch boundaries.

The $N_p$-dimensional vector property $\mathbf{m}$ 
\begin{align}\label{eq:prop3}
\mathbf{m} = & [m_0, \ldots, m_{N_p-1}]^T 
\end{align}
is the unknown vector of the inversion. Each patch $k$ has a constant density $m_k$.

### FEM domain
To set this up we first define a coarse grid in which we define patches and indicator functions $\chi_k$: 

In [2]:
NE0_coarse = 40  # number of coarse level cells in horizontal direction
NE1_coarse = 20  # number of coarse level cells in vertical direction
h_coarse = 2500. # grid spacing on coarse level [m]

Spatial extend of the domain based::

In [3]:
L0, L1= NE0_coarse*h_coarse, NE1_coarse*h_coarse
f"domain extend [m]: {L0} x  {L1}"

'domain extend [m]: 100000.0 x  50000.0'

With in this coarse level grid we define the patches in terms of patch grid dimension `PatchGrid_coarse` and
origin `PatchOrigin_coarse` which is the lower left corner of the patch grid: 

In [4]:
PatchGrid_coarse = (6,4)   # number of patches in x0 and x1 direction
PatchOrigin_coarse = NE0_coarse//2 - PatchGrid_coarse[0]//2, NE1_coarse//2 - PatchGrid_coarse[1] #

In [5]:
PatchOrigin_coarse

(17, 6)

The line of observers along the  we place two cells above the top of the patch array: 

In [6]:
ObserverHeight_coarse=PatchGrid_coarse[1]+PatchOrigin_coarse[1]+2

In [7]:
ObserverHeight_coarse

12

We introduce a refinement factor `Refine` so we can work with finer grid for the FEM solution. 
For instance a value `Refine=2` halfs each coarse level cell. 

In [8]:
Refine=4     # refinement factor 
DataStep=4   # use every `DataStep`-th element to grab data

Now we have the compuational grid with $\times \mathtt{Refine}$ more elements in each direction. The 
element size is reduced accordingly to keep the extension of the domain as set by the coarse grid:

In [9]:
NE0=NE0_coarse * Refine
NE1=NE1_coarse * Refine
h = h_coarse/Refine

As the refined domain the same extend as the original domain? 

In [10]:
f"domain extend [m]: {NE0*h} x  {NE1*h}"

'domain extend [m]: 100000.0 x  50000.0'

Now we can set up the `esys.finley` domain: 

In [11]:
from esys.escript import *
from esys.finley import Rectangle

domain=Rectangle(n0=NE0, n1=NE1, l0=L0, l1=L1)

### Definition of indicator functions

Setting the indicator for patch $(k_0,k_1)$ we introduce four masks (with values values `0` and `1`) which are then combined the to define the region of the patch:

- `m1` = has values `1` for elements above the bottom edge of the patch: $x_1 > (PatchOrigin\_coarse[1]+k1)\cdot h\_coarse$
- `m2` = has values `1` for elements below the top edge of the patch: $x_1 < (PatchOrigin\_coarse[1]+k1+1)\cdot h\_coarse$
- `m3` = has values `1` for elements to the right of left edge of the patch: $x_0 > (PatchOrigin\_coarse[0]+k0)\cdot h\_coarse$
- `m4` = has values `1` for elements to the left of right edge of the patch: $x_0 < (PatchOrigin\_coarse[0]+k0+1)\cdot h\_coarse$

In [12]:
chis=[]
X=ReducedFunction(domain).getX()
for k0 in range(PatchGrid_coarse[0]):  
    for k1 in range(PatchGrid_coarse[1]):
        m1=wherePositive(X[1]-(PatchOrigin_coarse[1]+k1)*h_coarse)
        m2=whereNegative(X[1]-(PatchOrigin_coarse[1]+k1+1)*h_coarse)
        m3=wherePositive(X[0]-(PatchOrigin_coarse[0]+k0)*h_coarse)
        m4=whereNegative(X[0]-(PatchOrigin_coarse[0]+k0+1)*h_coarse)
        chis.append(m1*m2*m3*m4)

Lets do little test and mark each patch `chi` by its index in the `chis` list :

In [13]:
rho=0
for k,chi in enumerate(chis):
    rho=rho+chi*(k+1)

In [14]:
rho_np=convertToNumpy(rho)
X_np=convertToNumpy(rho.getX())

plt.figure()
plt.tricontourf(X_np[0], X_np[1], rho_np[0], 25, cmap='tab20c')
plt.xlabel('$x_0$ [m]')
plt.ylabel('$x_1$ [m]')
plt.colorbar()

RuntimeError: unidentifiable C++ exception

###  Create a density distribution

Now we can create some test data $\mathbf{d}^{obs}$ from an assumed density distribution from a propert vector $\mathbf{m}_{true}$ using the patches:

In [ ]:
m_true=np.zeros(PatchGrid_coarse)
m_true[0,1]=1500
m_true[4,3]=-1000

Let's take a look at teh values over the patch. Notice the `.T` which is needed to swap the first and second dimension as `imshow` plot first dimension upward.

In [ ]:
plt.clf()
vmax=abs(m_true).max()
plt.imshow(m_true.T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()

Now we can build a density distribution from `m_true` on the FEM grid:

In [ ]:
rho=0
for k in range(m_true.size):
    rho=rho+chis[k]*m_true.flat[k]

Always good to check:

In [ ]:
rho_np=convertToNumpy(rho)
X_np=convertToNumpy(rho.getX())

plt.figure()

plt.tricontourf(X_np[0], X_np[1], rho_np[0], 15)
plt.xlabel('$x_0$ [m]')
plt.ylabel('$x_1$ [m]')
plt.colorbar()

### Compute the gravity field

To compute the gravity field due to this density distribution we solve a PDE as discussed earlier:

In [ ]:
from esys.escript.linearPDEs import LinearSinglePDE

model=LinearSinglePDE(domain)
x=domain.getX()
model.setValue(A=np.eye(2), q=whereZero(x[1]-L1))

Set the right hand side of PDE and solve:

In [ ]:
model.setValue(Y=-4*np.pi*G*rho)
u=model.getSolution()

and the vertical gravity at element centers:

In [ ]:
gz=-grad(u, ReducedFunction(domain))[1]

### Grab the data along a line

We use a `Locator` to grab the vertical gravity. These are the horizontal positions of element centers:

In [ ]:
x0_line=np.linspace(h/2, L0-h/2, NE0)

Add vertical position at $x_1=ObserverHeight\_coarse*h\_coarse$ and create the `Locator`:

In [ ]:
from esys.escript.pdetools import Locator
line_locator=Locator(where=gz.getFunctionSpace(), x=[ (x0, ObserverHeight_coarse*h_coarse) for x0 in x0_line ] )

Get the vertical gravity alonh the line:

In [ ]:
gz_line=np.array(line_locator.getValue(gz))*mGal

We take a subset of this as data (if `DataStep`>1). Notice the `copy` which is needed as the slicing creates a different view on `gz_line` only and we want to modify `d_obs` later.

In [ ]:
d_obs=gz_line[::DataStep].copy()

To make this more interesting we add 10\% random noise:

In [ ]:
d_obs*=(1+0.30*np.random.normal(scale=0.1, size=d_obs.shape))

What is $N_d$?

In [ ]:
f"number of data points N_d = {d_obs.size}."

In [ ]:
plt.figure()
plt.plot(x0_line, gz_line, label='true gz')
plt.scatter(x0_line[::DataStep], d_obs, s=2, label='data')
plt.xlabel('x [m]')
plt.ylabel('gravity [mGal]')
plt.legend()
plt.title("gravity anomaly over transect @ height %g m"%(line_locator.getX()[0][1]))

Check the misfit $\Phi_d$ of data vector $\mathbf{d}$ against the data assuming a constant error of $\sigma_i=0.5 mGal$:

In [ ]:
d=gz_line[::DataStep].copy()
phi_d=np.mean( (d-d_obs)**2/0.5**2)
phi_d

### Forward model in a compact form
For convenience we would like to capture the process we just implemented and express the construction of the data vector $\mathbf{d}$ from a property vector $\mathbf{m}$ in matrix. 

Recall that the FEM approximation 
of the gravity potential $u^{h}$ is the solution of the weak problem   
\begin{equation}\label{eq:GRAVINV1}
\int_{\Omega} \nabla v^{h}  \cdot \nabla u^{h}  \; dx =
\int_{\Omega} \rho v^{h}  \; dx 
\end{equation}
for all test functions $v^{h}$. For a given data vector $\mathbf{m}=[m_0,\ldots,m_{N_p-1}]^T$ 
the density is taking the form   
\begin{equation}\label{EQ:DENS2xb}
\rho = \sum_{k=0}^{N_p-1} m_k \chi_{k}
\end{equation}
as introduced earlier. With this we get
\begin{equation}\label{eq:GRAVINV2.1}
\int_{\Omega} \nabla v^{h}  \cdot \nabla u^{h}  \; dx =
\sum_{i=0}^{N_p-1}  m_k \int_{\Omega} \chi_k v^{h}  \; dx 
\end{equation}
for all test functions $v^{h}$. 

We can grab the values $\mathbf{U}^{h}$ of the FEM solution $u^{h}$ at the 
nodes at the FEM (duality of FEM node and basis). We express this in compact form and write
\begin{equation}
\mathbf{U}^{h} =\mathbf{L}^{h}[\mathbf{m}] \;. 
\end{equation}
So keep in mind that every time you see $\mathbf{L}^{h}[\mathbf{m}]$ we solve the FEM problem \eqref{eq:GRAVINV2.1}; this is the statement `u=model.getSolution()`.

Then we calculate vertical gravity $g_z$ at some element centers  $\mathbf{x}_i$ 
to obtain the data vector $\mathbf{d}$ from $\mathbf{U}^{h}$ or $u^h$. 
This was done by the statements:

    gz=-grad(u, ReducedFunction(domain))[1]
    gz_line=np.array(line_locator.getValue(gz))*mGal
    d=gz_line[::DataStep].copy()

In mathematical terms that means 
\begin{equation}
d_i=F_i(\mathbf{m})= g_z(\mathbf{x}_i) = - \left.\frac{\partial u^h}{\partial x_1}\right|_{\mathbf{x}=\mathbf{x}_i}
\end{equation}
As in the FEM space the gradient calculation uses FEM nodes values (see lectures) 
\begin{equation}\label{eq:forward2}
d_i = \mathbf{B}_i^T \mathbf{U}^{h}
\end{equation}
with a suitable vector $\mathbf{B}_i$ of length $N$ = number FEM nodes. So keep in mind that every time you see $\mathbf{B}_i^T \mathbf{U}$ this means to calculate the gradient of the FEM solution with values $\mathbf{U}$ and pick its value at point $\mathbf{x}_i$ as in the three lines of python code shown above.

We can put this two steps together to write the 
calculation of predicted observations $\mathbf{d}=[d_0, \ldots, d_{N_d-1}]^T$ in a compact matrix form
\begin{equation}\label{eq:forward3}
\mathbf{d} = \mathbf{B} \mathbf{L}^{h}[\mathbf{m}] 
\end{equation}
with $N_d \times N$ matrix $\mathbf{B}$ combining the evaluation of the 
vertical gravity from the FEM solution $\mathbf{u}^h$ at the observation points.

## Critical Point Condition

To find the solution of the minimization for $N_p$ real unknowns we use: 

*A necessary condition for a solution $\mathbf{m}^{*}$ of the minimization problem \eqref{eq:minimization}
is that $\mathbf{m}^{*}$ is a critical point of the cost function $\Phi_d$.*


That means
that at $\mathbf{m}^{*}$ all partial derivatives of $\Phi_d$ take the value zero at the same time:
\begin{equation}\label{eq:criticalpoint}
\left. \frac{\partial \Phi_d}{\partial m_k} \right|_{\mathbf{m}=\mathbf{m}^{*}}= 0 
\end{equation}
for all $k=0 \ldots N_p-1$. 

From the definition of the data misfit function one obtains
\begin{equation}\label{eq:criticalpoint2}
\frac{\partial \Phi_d}{\partial m_k} 
= \sum_{i=0}^{N_d-1} w^{obs}_i \cdot (F_i(\mathbf{m})-d_i^{obs}) 
\frac{\partial F_i}{\partial m_k}
\end{equation}
The key terms are $\frac{\partial F_i}{\partial m_k}$
which are assembled in so-called Jacobean matrix $\mathbf{J}=(J_{ik})$:
\begin{equation}\label{eq:jacobean}
J_{ik} = \frac{\partial F_i}{\partial m_k}
\end{equation}
$\mathbf{J}$ is an $N_d \times N_p$ matrix.

In matrix notation the critical point condition \eqref{eq:criticalpoint} is given as
\begin{equation}\label{eq:criticalpoint3}
\mathbf{J}^T \mathbf{W}_d (\mathbf{F}(\mathbf{m}^*)-\mathbf{d}^{obs}) =\mathbf{0}
\end{equation}
$\mathbf{J}$ is a measure of changes in the data predictions
due to changes in physical properties. It is also called sensitivity.

Question now is how do we calculate the Jacobean matrix for the gravity problem.

### Jacobean for the gravity problem

As matrix $\mathbf{B}_i$ (to grab vertical gravity at an observation point) 
is not depending on FEM 
solution $\mathbf{U}^h =\mathbf{L}^h[\mathbf{m}]$ one has
\begin{equation}\label{eq.FEMgradmatrix} 
J_{ik} = \frac{\partial F_i}{\partial m_k} = \mathbf{B}_i \cdot \frac{\partial \mathbf{U}^h}{\partial m_k}
\end{equation}
with $\frac{\partial \mathbf{U}^h}{\partial m_k} = 
\left[ \frac{\partial U^h_0}{\partial m_k}, \ldots, \frac{\partial U^h_{N-1}}{\partial m_k} \right]^T$. From 
the FEM basis representation 
\begin{equation}\label{eq.FEMinterpol3} 
u^h = \sum_{p=0}^{N-1} U^h_p \phi^h_p
\end{equation}
partial derivative of the FEM solution $u^h$ with respect to $m_k$ is given 
as
\begin{equation}\label{eq.FEMinterpol3.1} 
\frac{\partial u^h}{\partial m_k} = \sum_{p=0}^{N-1} \frac{\partial U^h_p}{\partial m_k} \phi^h_p
\end{equation}
We apply derivation respect of $m_k$ the to  weak from of the PDE \eqref{eq:GRAVINV1} for $u^h$:
\begin{equation}\label{eq:GRAV.DERIV1.1}
\int_{\Omega} \nabla v^{h}  \cdot \nabla \frac{\partial u^h}{\partial m_k}  \; dx =
\int_{\Omega} \frac{\partial \rho}{\partial m_k}  v^{h}  \; dx 
\end{equation}
for all $v^h$. 

From the expression for a piecewise constant density $\rho$ with coefficients $\mathbf{m}$ :
\begin{equation*}
\rho = \sum_{l=0}^{N_p-1} m_l \chi_{l}
\end{equation*}
one gets
\begin{equation}\label{eq:jacobean2}
\frac{\partial \rho}{\partial m_k} =  \sum_{l=0}^{N_p-1}  \delta_{lk} \chi_l = \chi_k
\end{equation}
which leads to
\begin{equation}\label{eq:GRAVINV1.1}
\int_{\Omega} \nabla v^{h}  \cdot \nabla u^{h;k}  \; dx =
\int_{\Omega} \chi_k  v^{h}  \; dx 
\end{equation}
with $\displaystyle{u^{h;k}=\frac{\partial u^h}{\partial m_k}}$. 
We collect the values of 
$u^{h;k}$ at the FEM nodes in vector $\mathbf{K}_k=\displaystyle{\frac{\partial \mathbf{U}^h}{\partial m_k}}$ 
and then can write Jacobean as 
\begin{equation}\label{eq.FEMgradmatrix1} 
J_{ik} = \mathbf{B}^T_i \cdot \mathbf{K}_k
\end{equation}
for $i=0,\ldots, N_d$ and $k=0,\ldots N_p$. 

We can easily implement this:

In [ ]:
J=np.zeros((d_obs.size, len(chis)))
for k in range(len(chis)):
    model.setValue(Y=-4*np.pi*G*chis[k])
    uk=model.getSolution() # = K_k
    gzk=-grad(uk, ReducedFunction(domain))[1]
    gzk_line=np.array(line_locator.getValue(gzk))*mGal 
    J[:,k]=gzk_line[::DataStep] # = B.K_k

In [ ]:
plt.figure()
for k in [0, 10, 20]:
    plt.plot(x0_line[::DataStep], J[:,k], label=f'col {k}')
plt.xlabel('x [m]')
plt.ylabel('gravity [mGal]')
plt.legend()
plt.title("Columns of the Jacobean")

### The linear model case 
The entries of matrix $\mathbf{K}_k$ are independent from $\mathbf{m}$ which also means that the forward problems $F_i$ are changing linear with the physical property vector. As a consequence we can use $\mathbf{K}_k$ also to calculate the FEM solution $u^h$ for a given property $\mathbf{m}$. 
The density $\rho$ is variational equation \eqref{eq:GRAVINV1} is written as 
\begin{equation}\label{eq:GRAVINV2}
\int_{\Omega} \rho v^{h}  \; dx = \sum_{k=0}^{N_p-1}  m_k \int_{\Omega} \chi_k v^{h}  \; dx 
= \sum_{k=0}^{N_p-1} m_k  \int_{\Omega} \nabla v^{h}  u^{h;k}  \; dx = 
\int_{\Omega} \nabla v^{h} \cdot \nabla  \left(\sum_{k=0}^{N_p-1} m_k u^{h;k} \right) \; dx
\end{equation}
for all test functions $v^{h}$. This shows that in fact 
\begin{equation}\label{eq:GRAFIT5}
u^{h}  = \sum_{i=0}^{N_p-1} m_k u^{h;k}
\end{equation}
is the FEM potential for a given  property vector $\mathbf{m}=[m_k]$.
This translates to nodal values in the form
\begin{equation}\label{eq:GRAFIT6}
\mathbf{U}^{h}  = \sum_{i=0}^{N_p-1} m_k \mathbf{K}_k
\end{equation}
and for the data vector $\mathbf{d}=[d_i]$
\begin{equation}\label{eq:GRAFIT8}
d_i = \mathbf{B}_i^T \mathbf{U}^{h} = \sum_{k=0}^{N_p-1} m_k \mathbf{B}_i^T \mathbf{K}_k 
 = \sum_{k=0}^{N_p-1} J_{ik} m_k 
\end{equation}
or 
\begin{equation}\label{eq:GRAFIT8.1}
\mathbf{d} = \mathbf{J} \mathbf{m}
\end{equation}
Notice that in general such identity applies only when the forward model is linear function of the
physical parameters such as the gravity and the magnetic but this will not apply for instance in the MT case.

With all this work we can restate the critical point condition \eqref{eq:criticalpoint2} can be written
\begin{equation}\label{eq:criticalpoint2.2}
(\mathbf{J}^T \mathbf{W}_d \mathbf{J}) \mathbf{m} = \mathbf{J}^T \mathbf{W}_d \mathbf{d}^{obs}
\end{equation}
This is a system of $N_p$  linear equations for the unknown property vector $\mathbf{m}$ with 
$N_p$ component. In the literature the equation is often referred as the normal equation of 
\begin{equation}\label{eq:criticalpoint2.3}
\mathbf{W}_d^{\frac{1}{2}} \mathbf{J} \mathbf{m} = \mathbf{W}_d^{\frac{1}{2}} \mathbf{d}^{obs}
\end{equation}
To make live a bit easier we assume that all measurement errors $\sigma_i$ are the same 
and hence the matrix $\mathbf{W}_d$ is the identity matrix.




## Solution

### First Attempt

There are a variety of solvers approaches available to solve these type of equations 
as provided by the python function `scipy.linalg.lstsq` in python, see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html).


In [ ]:
from scipy.linalg import lstsq
m_scipy, res, rnk, s=lstsq(J, d_obs)

And plot the result:

In [ ]:
plt.clf()
vmax=abs(m_scipy).max()
plt.imshow(m_scipy.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()

**Ops!** Values far to large! Wrong location of anomalies!

In [ ]:
m_scipy

In [ ]:
rho_scipy=0
for k in range(len(m_scipy)):
    rho_scipy=rho_scipy+chis[k]*m_scipy.flat[k]

In [ ]:
model.setValue(Y=-4*np.pi*G*rho_scipy)
u_scipy=model.getSolution()
gz_scipy=-grad(u_scipy, ReducedFunction(domain))[1]*mGal
d_scipy=np.array(line_locator.getValue(gz_scipy))[::DataStep].copy()

In [ ]:
plt.figure()
#plt.plot(x0_line,line_locator.getValue(gz_drop), label='gz')
plt.scatter(x0_line[::DataStep], d_obs, s=5, label='data')
plt.scatter(x0_line[::DataStep], d_scipy, s=5, label='recovered')

plt.xlabel('x [m]')
plt.ylabel('gravity [mGal]')
plt.legend()
plt.title("gravity anomaly over transect @ height %g m"%(line_locator.getX()[0][1]))

### SVD
To get a better understanding of what is going on we apply a singular value decomposition (SVD)
to the Jacobean $\mathbf{J}$.

What exactly is this an SVD? 

We try the respective function `numpy.linalg.svd` of `numpy`. It returns three arrays:


In [ ]:
U, s, V = np.linalg.svd(J, full_matrices=False)

Lets check shapes:

In [ ]:
U.shape, s.shape, V.shape, J.shape

Idea is that matrix `U` is an orthogonal matrix, that is $\mathbf{U}^T \mathbf{U} = \mathbf{I}$. 
Is this true?

In [ ]:
np.allclose(np.eye(24), U.T @ U)

The same is true for `V` - also an orthogonal matrix. Really?

In [ ]:
np.allclose(np.eye(24), V.T @ V)

The idea is that the matrices $\mathbf{U}$ and $\mathbf{V}$ transfer $\mathbf{J}$ into a diagonal matrix with
main diagonal given by array `s`. This is 
\begin{equation}\label{eq:SVD1}
\mathbf{U}^T \mathbf{J} \mathbf{V}^T = \mathbf{S} =
\begin{bmatrix}
s_0  & 0        &            & 0\\
    0      &  s_1&           & 0\\
    \vdots       &        & \ddots &    \vdots\\
        0         &       &       & s_{N_p-1}
\end{bmatrix}
\end{equation}
The values $s_0, \ldots, s_{N_p-1}$ are called the singular values of $\mathbf(J}$. Thry are always real and non-negative.

Is this working?

In [ ]:
np.allclose( np.diag(s), U.T @ J @ V.T)

Why is this useful? We can apply this to the critical point condition \eqref{eq:criticalpoint2.2}:  
\begin{equation}\label{eq:criticalpoint2.V2}
(\mathbf{J}^T \mathbf{J}) \mathbf{m} = \mathbf{J}^T \mathbf{d}^{obs}
\end{equation}
in the form
$$
\mathbf{U} \mathbf{S} \mathbf{V}= \mathbf{J}
$$
This gives
\begin{equation}\label{eq:criticalpoint2.V3}
(\mathbf{V}^T \mathbf{S} \mathbf{U}^T \mathbf{U} \mathbf{S} \mathbf{V}) \mathbf{m} = \mathbf{V}^T \mathbf{S} \mathbf{U}^T\mathbf{d}^{obs}
\end{equation}
We multiply by $ \mathbf{S}^{-1} \mathbf{V}$ and obtain
\begin{equation}\label{eq:criticalpoint2.V4}
 \mathbf{S} \mathbf{V} \mathbf{m} = \mathbf{U}^T\mathbf{d}^{obs}
\end{equation}
which gives solution
\begin{equation}\label{eq:criticalpoint2.V5}
\mathbf{m} =  \mathbf{V}^T \mathbf{S}^{-1} \mathbf{U}^T\mathbf{d}^{obs}
\end{equation}
Is this really the same solution as we got from `scipy.linalg.lstsq`?

In [ ]:
np.allclose(m_scipy, np.dot((V.T*1/s) @ U.T, d_obs), atol=0.)

### Some Analysis

We set $\hat{\mathbf{d}}=\mathbf{U}^T\mathbf{d}^{obs}$ which is is essence is just a change of variables in the space of the observations. With the rows $\mathbf{v}_0, \ldots, \mathbf{v}_{N_p-1}$ of matrix $\mathbf{V}$ (also called modes)
we then write the solution $\mathbf{m}$ in the following form: 
\begin{equation}\label{eq:svn4PASS}
\mathbf{m} =  \sum_{k=0}^{N_p-1} \frac{\hat{d}_k}{s_k} \mathbf{v}_k
\end{equation}
We see that the solution is a linear combinations of the rows of $\mathbf{V}$ where 
rows that belong to small singular values make the largest contributions - if the respective data contribution $\hat{d}_k$ is not zero as we can assume for data with noise.

Let's check the singular values (they are always returned in the order increasing value): 

In [ ]:
s[0]/s[-1]*1e-11, s[1], s[-1]

We see that the smalles singular value $ \approx 10^{-13}$ is extremely small compared to the largerest $10^{-1}$ which means that the solution is dominated by $\bathbf{v}_{23}$.

How does this look like?

In [ ]:
m=V[-1,:]
plt.clf()
vmax=abs(m).max()
plt.imshow(m.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for s={s[-1]}")

In [ ]:
m=V[-2,:]
plt.clf()
vmax=abs(m).max()
plt.imshow(m.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for s={s[-2]}")

These look very similar to what the solver returned. 

Let's test some of the other modes:

In [ ]:
m=V[2,:]
plt.clf()
vmax=abs(m).max()
plt.imshow(m.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for s={s[2]}")

In [ ]:
m=V[10,:]
plt.clf()
vmax=abs(m).max()
plt.imshow(m.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for s={s[1]}")

The vector $\mathbf{V} \mathbf{m}_{true}$ shows us which modes give  the largest contribution to the true solution:

In [ ]:
np.dot(V, m_true.flat)

Largest contribution from s[13]:

In [ ]:
m=V[13,:]
plt.clf()
vmax=abs(m).max()
plt.imshow(m.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for s={s[13]}")

What did we learn?

- The large ratio of largest to smallest singular value leads to over emphasising modes corresponding to the smallest singular values. This characteristic is called ill-posedness. 
- In ill-post problems noise in the input can result in solutions that are far off from the true solution.
- inversion is an ill-posed problem.

So what can we do about this?

## Solution Strategies

There are  a few thing one can try to address this issue. 
It always comes at some costs in terms of the accuracy of the returned solution.


### Mode dropping

If a SVD of the Jacobean $\mathbf{J}$ is available a simple strategy is to suppress contributions from the 
low singular values. This approach is motivated by the idea that these 
small singular values should (maybe) actually be considered as zeros.
This concept of a high-pass filter is formally introduced by introducing
weighting factors factors $\varrho_k$ into 
the calculation of the solution $\mathbf{m}$ in \eqref{eq:svn4PASS}
 \begin{equation}\label{eq:svn4PASS DROP}
\mathbf{m} = \sum_{i=0}^{N_p-1} \varrho_k \cdot \frac{\hat{d}_k}{s_{k}} \mathbf{v}_k
\end{equation}
the idea being that $\varrho_k$ takes small values or set to zero 
for those singular values $s_k$ that are supposed to be suppressed. 
The simplest approach is simply switch off 
the *nasty* terms in \eqref{eq:svn4PASS} that is to 
ignore all terms for singular values less than
given threshold $\tau$ that means
we set  
\begin{equation}\label{eq:filter}
\varrho_k =
 \left\{ 
\begin{array}{cl}
1 & s_k > \tau  \\
0 & \mbox{ otherwise }
\end{array}
\right.
\end{equation}
This acts as a discriminating high-pass filter.
It introduces a deviation from the original 
problem \eqref{eq:criticalpoint2.2} to be solved but dumps the components that introduce ill-posedness.
The larger value for the cut off  $\tau$ is chosen 
the better the problem becomes conditioned but at the same time the
larger the deviation from the original problem get. So a value needs to be chosen that provides a balance between these two conflicting targets. Typically one chooses 
$$
\tau = \alpha \cdot \underset{k}{max} \; s_k
$$
with for instance $\alpha = 0.001$.

We put this to the test:

First find the index `drop` at which the 

In [ ]:
alpha=5e-3
drop=np.argmax(s<alpha*max(s)) # returns index of first True
drop

In [ ]:
d_hat=np.dot(U.T, d_obs)

In [ ]:
m_drop=np.dot(V.T[:,:drop], d_hat[:drop]/s[:drop])

In [ ]:
plt.clf()
vmax=abs(m_drop).max()
plt.imshow(m_drop.reshape(PatchGrid_coarse).T, origin='lower',  vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for with drop tolerance alpha={alpha}")

What about the misfit?

In [ ]:
rho_drop=0
for k in range(len(m_drop)):
    rho_drop=rho_drop+chis[k]*m_drop.flat[k]

In [ ]:
model.setValue(Y=-4*np.pi*G*rho_drop)
u_drop=model.getSolution()
gz_drop=-grad(u_drop, ReducedFunction(domain))[1]*mGal
d_drop=np.array(line_locator.getValue(gz_drop))[::DataStep].copy()

In [ ]:
sqrt(np.mean((d_drop-d_obs)**2)) ,sqrt(np.mean((d_obs)**2))

In [ ]:
plt.figure()
#plt.plot(x0_line,line_locator.getValue(gz_drop), label='gz')
plt.scatter(x0_line[::DataStep], d_obs, s=5, label='data')
plt.scatter(x0_line[::DataStep], d_drop, s=5, label='recoverd')

plt.xlabel('x [m]')
plt.ylabel('gravity [mGal]')
plt.legend()
plt.title("gravity anomaly over transect @ height %g m"%(line_locator.getX()[0][1]))

### Filtering

A discriminating approach for a high-pass filter may be a bit harsh
as modes can only be switched on or off and so changes to the cut off could lead to significant changes in the result. A more appropriate strategy could be a continuous transition between zero to switch of 
small singular values and one to keep contributions from modes for large singular values. For instance one can set 
 \begin{equation}\label{eq:svn4.1}
\varrho_k = \frac{ s_k^2}{s_k^2+\tau^2}
\end{equation}
This looks like
<img src="./Data/IMAGE_filter_function.png" width="400">
(soft) cut-offs $\tau$. Notice the log scale on the $s$ axis.
This selection is weighting factors is referred to as Wiener weights.
Singular values lower than $\tau$ receive a
weighting $\varrho$ less than $\frac{1}{2}$. Obviously 
there are other options to introduce a smooth 
high-pass filter.


In [ ]:
tau=1.e-2*max(s)
m_f=np.dot(V.T, d_hat/s * s**2/(s**2+tau**2))

Would be interesting to see the contribution of modes:

In [ ]:
plt.figure()
plt.plot(s, s/(s**2+tau**2), label="filtered")
plt.plot(s, 1/s, label="original")
plt.xlabel("s")
plt.ylabel("filtered")
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
plt.clf()
vmax=abs(m_f).max()
plt.imshow(m_f.reshape(PatchGrid_coarse).T, origin='lower', vmin=-vmax, vmax=vmax, cmap='seismic')
plt.colorbar()
plt.title(f"mode for Wiener filter tau={tau:.2g}")

In [ ]:
rho_f=0
for k in range(len(m_f)):
    rho_f=rho_f+chis[k]*m_f.flat[k]

In [ ]:
model.setValue(Y=-4*np.pi*G*rho_f)
u_f=model.getSolution()
gz_f=-grad(u_f, ReducedFunction(domain))[1]*mGal
d_f=np.array(line_locator.getValue(gz_f))[::DataStep].copy()

In [ ]:
sqrt(np.mean((d_f-d_obs)**2)), sqrt(np.mean((d_obs)**2))

In [ ]:
plt.figure()
#plt.plot(x0_line,line_locator.getValue(gz_drop), label='gz')
plt.scatter(x0_line[::DataStep], d_obs, s=8, label='data')
plt.scatter(x0_line[::DataStep], d_f, s=5, label='recoverd')

plt.xlabel('x [m]')
plt.ylabel('gravity [mGal]')
plt.legend()
plt.title("gravity anomaly over transect @ height %g m"%(line_locator.getX()[0][1]))

## Regularization

Another approach is going back to the inversion problem and modify the objective function to be minimized. The idea is to bound the values property function $\mathbf{m}$ and add a penalty term to the misfit. So instead 
of minimizinf $\Phi_d$ we now minimize now
$$
\Phi(\mathbf{m}) =  \Phi_d(\mathbf{m}) + \mu \frac{1}{2}  \mathbf{m}^T\mathbf{m}
$$
for some positive penalty factor $\mu$. This approach is called regularization and 
the extra term is called the regularization term. This approach is attractive as we do not need 
a SVD of $\mathbf{J}$. 

Again the solution $\mathbf{m}$ is given as a critical point which 
means that all partial derivatives of $\Phi$ withe respect to $m_k$ take the value zero at the same time:
\begin{equation}\label{eq:criticalpoint R}
\left. \frac{\partial \Phi}{\partial m_k} \right|_{\mathbf{m}=\mathbf{m}^{*}}= 0 
\end{equation}
for all $k=0 \ldots N_p-1$. This leads to the condition 
\begin{equation}\label{eq:criticalpoint2R}
\frac{\partial \Phi}{\partial m_k} 
= \mu \cdot m_k +\sum_{i=0}^{N_d-1} (F_i(\mathbf{m})-d_i^{obs})  =
\frac{\partial F_i}{\partial m_k}
\end{equation}
Bringing in the Jacobean matrix $\mathbf{J}=(J_{ik})$
\begin{equation}\label{eq:criticalpoint3R}
\mu \mathbf{m}^* + \mathbf{J}^T (\mathbf{J}\mathbf{m}^*-\mathbf{d}^{obs}) =\mathbf{0}
\end{equation}
which we can write in the form (dropping the upper index $*$)
\begin{equation}\label{eq:criticalpoint3RS}
( \mu \mathbf{I} + \mathbf{J}^T \mathbf{J}) \mathbf{m} = \mathbf{J}^T \mathbf{d}^{obs}
\end{equation}
This can easily be solved by inverting the matrix $\mu \mathbf{I} + \mathbf{J}^T \mathbf{J}$.
For $\mu=0$ we are back to the original ill-posed problem. Did we really gain anything?


From the SVD compostion we have:
$$
\mu \mathbf{I} + \mathbf{J}^T \mathbf{J} = 
\mathbf{V}^T(\mu \mathbf{I} +  \mathbf{S}^2) \mathbf{V}
$$
as $\mathbf{V}$ is orthogonal. Hence we have 
$$
(\mu \mathbf{I} + \mathbf{J}^T \mathbf{J})^{-1} = 
\mathbf{V}^T(\mu \mathbf{I} + \mathbf{S}^2)^{-1} \mathbf{V}
=\mathbf{V}^T \mathbf{D}^{-1}\mathbf{V}
$$
with diagonal matrix
$$
\mathbf{D} =
\begin{bmatrix}
\mu+s_0^2  & 0        &            & 0\\
    0      &  \mu+s_1^2&           & 0\\
    \vdots       &        & \ddots &    \vdots\\
        0         &       &       & \mu+s_{N_p-1}^2
\end{bmatrix}
$$
And right hand side is then
$$
\mathbf{J}^T \mathbf{d}^{obs}=\mathbf{V}^T \mathbf{S}  \mathbf{U}^T \mathbf{d}^{obs} = 
\mathbf{V}^T \mathbf{S} \hat{\mathbf{d}}^{obs}
$$
So the solution $\mathbf{m}$ is given as
$$
\mathbf{m} = \mathbf{V}^T \mathbf{D}^{-1} \mathbf{S}\hat{\mathbf{d}}^{obs} 
$$
What is $\mathbf{D}^{-1} \mathbf{S}$?
$$
\mathbf{D}^{-1} \mathbf{S} =
\begin{bmatrix}
\frac{s_0}{\mu+s_0^2}  & 0        &            & 0\\
    0      &  \frac{s_1}{\mu+s_1^2}&           & 0\\
    \vdots       &        & \ddots &    \vdots\\
        0         &       &       & \frac{s_0}{\mu+s_{N_p-1}^2}
\end{bmatrix}
$$
Do you recognize this?

This is the same as the Wiener filter:
$$
[\mathbf{D}^{−1}\mathbf{S}]_{kk}=\frac{s_0}{\mu+s_0^2} =\frac{\varrho_k}{s_k}
$$
if we set $\mu=\tau^2$.

Let's put is to the test:

In [ ]:
mu=tau**2

Get the solution $\matbf{m}_r$ from solving with regularization
$$
\mathbf{m}_r=( \mu \mathbf{I} + \mathbf{J}^T \mathbf{J})^{-1} \mathbf{J}^T \mathbf{d}^{obs}
$$
and then compare with the solution $\mtahbf{m}_f$ with Weiner filtering:

In [ ]:
m_r=np.linalg.solve(mu*np.eye(s.size)+J.T @ J, np.dot(J.T, d_obs))

In [ ]:
np.allclose(m_f, m_r)

Excellent we don't need the SVD but value $\mu$ becomes guesswork then.

## L-curves

A way to identify a good regularization parameter is plotting the values of the misfit $\Phi_p$ versus the regularization term (without the $\mu$ factor). One is looking for the point when further increase of the $\mu$ enforcing a more regular $\mathbf{m}$ starts to make a significant impact on an increase of the mi

In [ ]:
fit=[]
reg=[]
qs=[]
for q in np.linspace(-7,0,50):
    mu=10**q
    m_r=np.linalg.solve(mu*np.eye(s.size)+J.T @ J, np.dot(J.T, d_obs))
    rho_r=0
    for k in range(len(m_r)):
        rho_r=rho_r+chis[k]*m_r[k]
    model.setValue(Y=-4*np.pi*G*rho_r)
    u_r=model.getSolution()
    gz_r=-grad(u_r, ReducedFunction(domain))[1]*mGal
    d_r=np.array(line_locator.getValue(gz_r))[::DataStep].copy()
    fit.append(np.mean((d_r-d_obs)**2))
    reg.append(np.dot(m_r,m_r)/1000.)
    qs.append(q)
    

In [ ]:
plt.figure()
plt.plot(reg, fit)
plt.scatter(np.dot(m_f,m_f)/1000,np.mean((d_f-d_obs)**2))
for i, q in enumerate(qs):
    plt.annotate(f"{q:.1f}", (reg[i], fit[i]))
plt.xlabel("regularization")
plt.ylabel("misfit")
#plt.yscale("log")